# 微分可能LUTモデルによるMNIST学習

Stochasticモデルに BatchNormalization や Binarize(backward時はHard-Tanh)を加えることで、より一般的なデータに対してLUT回路学習を行います。 

## 事前準備

In [13]:
import os
import shutil
import numpy as np
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

device_id = 1
bb.set_device(device_id)
print(f"BinaryBrain : {bb.get_version_string()}")
print(f"Device : {bb.get_device_name(device_id)}")

BinaryBrain : 4.4.0
Device : NVIDIA GeForce GT 1030


異なる閾値で2値化した画像でフレーム数を水増ししながら学習させます。この水増しをバイナリ変調と呼んでいます。

ここではフレーム方向の水増し量を frame_modulation_size で指定しています。

In [14]:
# configuration
data_path             = './data/'
net_name              = 'MnistDifferentiableLut4Simple'
data_path             = os.path.join('./data/', net_name)
rtl_sim_path          = '../../verilog/mnist/tb_mnist_lut_simple'
rtl_module_name       = 'MnistLutSimple'
output_velilog_file   = os.path.join(data_path, rtl_module_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')

epochs                = 10
mini_batch_size       = 64
frame_modulation_size = 15

データセットは PyTorch の torchvision を使います。ミニバッチのサイズも DataLoader で指定しています。
BinaryBrainではミニバッチをフレーム数として FrameBufferオブジェクトで扱います。
バイナリ変調で計算中にフレーム数が変わるためデータセットの準備観点でのミニバッチと呼び分けています。

In [15]:
# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

## ネットワークの構築

DifferentiableLut に特に何もオプションをつけなければOKです。<br>
バイナリ変調を施すためにネットの前後に RealToBinary層とBinaryToReal層を入れています。<br>
send_command で "binary true" を送ることで、DifferentiableLut の内部の重み係数が 0.0-1.0 の間に拘束されます。

接続数がLUTの物理構成に合わせて、1ノード当たり6個なので層間で6倍以上ノード数が違うと接続されないノードが発生するので、注意してネットワーク設計が必要です。
最終段は各クラス7個の結果を出して Reduce で足し合わせています。こうすることで若干の改善がみられるとともに、加算結果が INT3 相当になるために若干尤度を数値的に見ることができるようです。

In [16]:
# define network
net = bb.Sequential([
            bb.RealToBinary(frame_modulation_size=frame_modulation_size),
            bb.DifferentiableLut([64*4*2], connection='random', N=4),
            bb.DifferentiableLut([64*4],   connection='serial', N=4),
            bb.DifferentiableLut([64],     connection='serial', N=4),
            bb.DifferentiableLut([10*4*4], connection='random', N=4),
            bb.DifferentiableLut([10*4],   connection='serial', N=4),
            bb.AverageLut       ([10],     connection='serial', N=4),
            bb.BinaryToReal(frame_integration_size=frame_modulation_size)
        ])
net.set_input_shape([1, 28, 28])

net.send_command("binary true")

loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam()

optimizer.set_variables(net.get_parameters(), net.get_gradients())

## 学習の実施

load_networks/save_networks で途中結果を保存/復帰可能できます。ネットワークの構造が変わると正常に読み込めなくなるので注意ください。
(その場合は新しいネットをsave_networksするまで一度load_networks をコメントアウトください)

tqdm などを使うと学習過程のプログレス表示ができて便利です。

In [17]:
#bb.load_networks(data_path, net)

# learning
for epoch in range(epochs):
    # learning
    loss.clear()
    metrics.clear()
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)
            
            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)
            
            net.backward(dy_buf)

            optimizer.update()
            
            t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))

    bb.save_networks(data_path, net)

  0%|          | 0/938 [00:00<?, ?it/s]

epoch[0] : loss=1.693157 accuracy=0.754300


  0%|          | 0/938 [00:00<?, ?it/s]

epoch[1] : loss=1.692850 accuracy=0.745600


  0%|          | 0/938 [00:00<?, ?it/s]

...

epoch[7] : loss=1.672507 accuracy=0.776900


  0%|          | 0/938 [00:00<?, ?it/s]

epoch[8] : loss=1.669994 accuracy=0.782100


  0%|          | 0/938 [00:00<?, ?it/s]

epoch[9] : loss=1.684189 accuracy=0.778100


## FPGA用Verilog出力

最後に学習したネットワークを Verilog 出力します。
MNISTのサイズである 28x28=784bit の入力を 10bit の分類をして出力するだけのシンプルなモジュールを出力します。

In [18]:
# export verilog
bb.load_networks(data_path, net)

# 結果を出力
with open(output_velilog_file, 'w') as f:
    f.write('`timescale 1ns / 1ps\n\n')
    bb.dump_verilog_lut_layers(f, module_name=rtl_module_name, net=net, device="")

# Simulation用ファイルに上書きコピー
os.makedirs(rtl_sim_path, exist_ok=True)
shutil.copyfile(output_velilog_file, sim_velilog_file)

'../../verilog/mnist/tb_mnist_lut_simple/MnistLutSimple.v'

In [19]:
# シミュレーション用データファイル作成
with open(os.path.join(rtl_sim_path, 'mnist_test.txt'), 'w') as f:
    bb.dump_verilog_readmemb_image_classification(f ,loader_test)

## モデルの内部の値を取得する

Verilog以外の言語やFPGA以外に適用したい場合、接続とLUTテーブルの2つが取得できれば同じ計算をするモデルをインプリメントすることが可能です。

### 事前準備
そのままだと勾配はリセットされているので少しだけ逆伝搬を実施します

In [20]:
# 最新の保存データ読み込み
bb.load_networks(data_path, net)

# layer を取り出す
layer0 = net[1]
layer1 = net[2]
layer2 = net[3]

### 接続を取得する

LUTモデルは get_connection_list() にて接続行列を取得できます。<br>
ここでの各出力ノードは、4つの入力と接続されており、layer0 の出力ノードは 256 個あるので、256x4 の行列が取得できます。

In [21]:
connection_mat = np.array(layer0.get_connection_list())
print(connection_mat.shape)
connection_mat

(512, 4)


array([[531, 302, 316, 420],
       [736, 111,  73, 191],
       [529, 203, 400,  20],
       ...,
       [770, 570, 749, 161],
       [633, 361, 153, 380],
       [221, 664, 145, 316]])

### FPGA化する場合のLUTテーブルを取得する

LUT化する場合のテーブルを取得します。<br>
4入力のLUTモデルなので $ 2^4 = 16 $ 個のテーブルがあります。<br>
モデル内に BatchNormalization 等を含む場合はそれらも加味して最終的にバイナリLUTにする場合に適した値を出力します。

In [22]:
lut_mat = np.array(layer0.get_lut_table_list())
print(lut_mat.shape)
lut_mat

(512, 16)


array([[False, False, False, ..., False,  True,  True],
       [False,  True, False, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True, False, False, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ...,  True,  True,  True]])

### 重み行列を覗いてみる

4入力のLUTモデルなので $ 2^4 = 16 $ 個のテーブルがあります。<br>
W() にて bb.Tensor 型で取得可能で、numpy() にて ndarray に変換できます。

In [23]:
W = layer0.W().numpy()
print(W.shape)
W

(512, 16)


array([[0.5465039 , 0.51769197, 0.405086  , ..., 0.49462226, 0.64438874,
        0.62060916],
       [0.4031878 , 0.5013975 , 0.40114996, ..., 0.58109206, 0.5712717 ,
        0.5900992 ],
       [0.45296904, 0.584766  , 0.68421924, ..., 0.5706679 , 0.4840127 ,
        0.42588487],
       ...,
       [0.6289483 , 0.47644472, 0.47481284, ..., 0.46310887, 0.43742472,
        0.40846857],
       [0.50208795, 0.51289016, 0.4730616 , ..., 0.38273937, 0.41233477,
        0.4111044 ],
       [0.44781703, 0.5007906 , 0.49563143, ..., 0.6021698 , 0.5975893 ,
        0.6541212 ]], dtype=float32)

### 勾配を覗いてみる

同様に dW() でW の勾配が取得できます

In [24]:
# そのままだとすべて0なので、1回だけbackward実施
for images, labels in loader_test:
    x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
    y_buf = net.forward(x_buf, train=True)
    net.backward(loss.calculate(y_buf, t_buf))
    break

dW = layer0.dW().numpy()
print(dW.shape)
dW

(512, 16)


array([[ 5.7463639e-04,  1.9154529e-04, -5.0615228e-05, ...,
        -4.5204455e-05, -3.9065577e-05, -1.3021869e-05],
       [-9.3714334e-09, -3.0122465e-09, -3.0122465e-09, ...,
         9.4587449e-10,  9.4587449e-10,  3.1877789e-10],
       [ 1.2546233e-03,  4.1481134e-04,  3.9722130e-05, ...,
         1.3524250e-04, -8.3377893e-04, -2.8811570e-04],
       ...,
       [-1.0291720e-04, -3.6128680e-05, -1.7183146e-04, ...,
         2.5349545e-05,  1.5734731e-05,  5.1773677e-06],
       [ 8.9322717e-04,  2.9068897e-04,  2.9114884e-04, ...,
        -4.6265649e-04,  1.0927854e-04, -1.7399911e-04],
       [ 8.9202821e-04,  2.9734289e-04,  1.6766042e-04, ...,
        -6.8563328e-05, -1.4753768e-04, -4.9179245e-05]], dtype=float32)